In [1]:
%pdb
from os.path import join
import pandas as pd

Automatic pdb calling has been turned ON


In [2]:
forecast_start_date = "5/6/20"
forecast_end_date = "6/21/20"
regions =['delhi', 'mumbai', 'ahmedabad', 'jaipur', 'bengaluru urban', 'pune']

folder = "/Users/namratadeka/covid19-india/covid19-library/src/results"
old_forecast_files ={
                        'delhi': join(folder, "forecast_delhi_5-6-20_6-21-20_2020-05-13 14:22:11.518287.csv"),
                        'mumbai': join(folder, "forecast_mumbai_5-6-20_6-21-20_2020-05-13 14:25:17.595065.csv"),
                        'ahmedabad': join(folder, "forecast_ahmedabad_5-6-20_6-21-20_2020-05-13 14:32:04.876977.csv"),
                        'jaipur': join(folder, "forecast_jaipur_5-6-20_6-21-20_2020-05-13 14:17:45.610277.csv"),
                        'bengaluru urban': join(folder, "forecast_bengaluru urban_5-6-20_6-21-20_2020-05-13 14:12:35.186261.csv"),
                        'pune': join(folder, "forecast_pune_5-6-20_6-21-20_2020-05-13 15:06:37.222578.csv")
                    }

#TODO: fix these numbers for all the regions
test_MAPEs = {
                'delhi': 9.83835079725655,
                'mumbai': 17.46514666428454,
                'ahmedabad': 2.6415996733971943,
                'jaipur': 1.9731123608687775,
                'bengaluru urban': 6.280067603602904,
                'pune': 10.897526315232458
             }

raw_csv ={
            'delhi': join(folder, "raw_data_delhi_state.csv"),
            'mumbai': join(folder, "raw_data_mumbai_district.csv"),
            'ahmedabad': join(folder, "raw_data_ahmedabad_district.csv"),
            'jaipur': join(folder, "raw_data_jaipur_district.csv"),
            'bengaluru urban': join(folder, "raw_data_bengaluru urban_district.csv"),
            'pune': join(folder, "raw_data_pune_district.csv")
        }

In [3]:
# these are numbers on the start date - may 5th or so
# counts of F bucket - recovered +deceased count on start date - actual
final_counts_on_start = dict()
for region in regions:
    raw = pd.read_csv(raw_csv[region], index_col='index')
    final_counts_on_start[region] = float(raw.loc[forecast_start_date]['recovered']) + float(raw.loc[forecast_start_date]['deceased'])

final_counts_on_start

{'delhi': 679.0,
 'mumbai': 1263.0,
 'ahmedabad': 834.0,
 'jaipur': 413.0,
 'bengaluru urban': 50.0,
 'pune': 435.0}

In [4]:
# global numbers for now
C_fatalities = 0.1  # breakup of F into R and D
C_hosp_stay_final = 0.068 #T_hosp = 14.5 days - breakup of Rbig into H and F 
C_icu =0.02 # computing icu as fraction of hospitalization



In [5]:
dfs = dict()
for region in regions: 
    infile = old_forecast_files[region]
    D = pd.read_csv(infile,header=0,delimiter=',').copy()

    #Rbig old was split into recovered and deceased, but the semantics aren't right
    # It is actually H + F and F -breaks into recovered and deceased  
    D['R_big'] = D['recovered_mean'] +D['deceased_mean']
    
    
    # Splitting Rbig into H and F
    D['F']=0
    D['H'] = 0
    D.at[0,'F'] = final_counts_on_start[region] # actuals
    D.at[0,'H'] = D.at[0,'R_big'] - D.at[0,'F']

    # i is both row numeric id and the index
    for i in range(len(D)) :   
        if i < (len(D)-1):
            D.at[(i+1), 'F'] = D.at[i,'F'] + C_hosp_stay_final* D.at[i,'H']
            D.at[(i+1), 'H'] = D.at[(i+1),'R_big'] - D.at[(i+1), 'F']

    #I - old active
    #E - old exposed

    # disjoint fractions of final bucket
    D['recovered_mean'] = D['F']*(1-C_fatalities)
    D['deceased_mean'] = D['F']*C_fatalities

    D['hospitalized_mean'] =D['H']
    # fraction of hospitalization
    D['icu_mean'] = D['H']*C_icu

    # new active = I + H  -  exposed is undetected - need to do this carefully 
    D['active_mean'] = D['active_mean'] +D['H'] 
    # new confirmed =  I + H + F 
    D['confirmed_mean'] =D['active_mean'] +D['F']

    #TODO - compute the min, max based on MAPES
    # a  x*(1+a),  x*(1-a) 
    # Forecast module
    cols = ['recovered', 'deceased', 'hospitalized', 'icu', 'active', 'confirmed']
    error = float(test_MAPEs[region] / 100)
    for col in cols:
        D['{}_min'.format(col)] = D['{}_mean'.format(col)].multiply(1-error)
        D['{}_max'.format(col)] = D['{}_mean'.format(col)].multiply(1+error)
    
    dfs[region] = D
    D.to_csv(join(folder, "corrected_forecast_{}_{}.csv".format(region, forecast_end_date.replace('/', '-'))))
    
    

In [6]:
dfs

{'delhi':       index      exposed  active_mean  recovered_mean  deceased_mean  \
 0    5/6/20  1853.220916  4767.133586           611.1           67.9   
 1    5/7/20  2016.891843  5097.113200           663.3           73.7   
 2    5/8/20  2091.576804  5395.149101           765.9           85.1   
 3    5/9/20  2107.983026  5657.852875           909.0          101.0   
 4   5/10/20  2089.268580  5881.650450          1085.4          120.6   
 5   5/11/20  2046.016613  6069.300835          1288.8          143.2   
 6   5/12/20  1987.816983  6222.873518          1514.7          168.3   
 7   5/13/20  1922.246023  6341.070941          1759.5          195.5   
 8   5/14/20  1851.740380  6430.283552          2019.6          224.4   
 9   5/15/20  1778.626568  6491.064177          2291.4          254.6   
 10  5/16/20  1705.818945  6527.221745          2573.1          285.9   
 11  5/17/20  1634.556552  6538.756623          2862.9          318.1   
 12  5/18/20  1564.660251  6529.527300    